![iscap_logo](https://www.iscap.ipp.pt/logo-ipp.png)

# Trabalho Prático

O objetivo do trabalho é desenvolver um programa para fazer a gestão de alunso numa instituição de ensino. O programa não necessita de ter interface gráfica, podendo apenas funcionar à base de inputs como implementado no código de exemplo.

### As funcionalidades obrigatórias são:
* Permitir visualizar todos os alunos
* Permitir criar novos alunos por upload de csv (exemplo fornecido) e via input()
* Permitir editar / apagar alunos
* Permitir exportar os dados de alunos para excel
* Permitir adicionar / apagar unidades curriculares
* Visualizar estatisticas básicas como:
    * Número de alunos
    * Mínimo, média, máximo de nota (geral e por unidade curricular)
    * Número de alunos aprovados (geral e por unidade curricular)
    * ...
* Bónus: implementar pelo menos mais uma funcionalidade extra, idealizada pelo grupo. Máximo de 1.5 valores extra para funcionalidades criativas e com complexidade adicional.

### Funcionamento base:
* Sempre que o programa é iniciado (correndo a função 'run') deverá verificar se tem um ficheiro no sistema com o nome "database.csv", se sim deverá ler o ficheiro e assumir esses registos por default.
* Ao sair do programa, este deverá questionar o user se quer guardar as alterações, se sim o programa deverá fazer overwrite do ficheiro "database.csv".
* Ao inserir novos alunos deverá apenas aceitar caso os dados os nomes estejam na lista de novos possiveis (permitido usar a função 'fetch_list_from_txt'). Além disso, os nomes devem ser sempre guardados com a primeira letra em maiúsculas e as restantes em minúsculas. As restantes colunas devem ser todas convertidas para minúsculas. Estas condições são válidas tanto para a inserção manual como via ficheiro.
* Internamente a representação da informação dos aluns deverá ser guardada num pandas DataFrame.

### Identificação do grupo
| nome | número aluno |
|------|--------------|
|Ana Rita Maganinho|2180332|
|Bárbara Pedrosa|2240510|
|Liliana Gestosa|2240516|

### Student Management System

In [ ]:
import requests
from os.path import exists
import pandas as pd
from google.colab import files

In [ ]:
# URLs dos arquivos com os nomes e sobrenomes
FILE_FIRST_NAMES = "https://raw.githubusercontent.com/Tremocex/ALL_NAMES_PORTUGAL/main/names_final.txt"
FILE_SURNAMES = "https://raw.githubusercontent.com/Tremocex/ALL_NAMES_PORTUGAL/main/surnames_final.txt"

# Funções úteis
def fetch_list_from_txt(url: str) -> list:
    """Fetch data from a remote TXT file and convert to a list."""
    txt = requests.get(url).text
    return list(set(txt.split("\n")))

def exists_database(file_path: str = "database.csv") -> bool:
    """Check if a file with the name 'file_path' exists in the working dir."""
    return exists(file_path)

# Carregar listas de nomes e sobrenomes
first_names_list = fetch_list_from_txt(FILE_FIRST_NAMES)
surnames_list = fetch_list_from_txt(FILE_SURNAMES)
first_names_set = {name.capitalize() for name in first_names_list}
surnames_set = {surname.capitalize() for surname in surnames_list}

In [ ]:
def upload_csv_file() -> pd.DataFrame:
    """Allows the user to upload a CSV file directly in Google Colab."""
    print("Please upload the CSV file.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return pd.DataFrame()

    file_name = list(uploaded.keys())[0]
    print(f"File {file_name} uploaded successfully.")

    df = pd.read_csv(file_name)

    # Ensure first_name and surname are capitalized
    if 'first_name' in df.columns:
        df['first_name'] = df['first_name'].str.capitalize()
    if 'surname' in df.columns:
        df['surname'] = df['surname'].str.capitalize()

    print("File loaded into a DataFrame:")
    print(df.head())
    return df

def load_batch_students_via_upload(df):
    """Handles the process of uploading and loading batch students into the DataFrame."""
    uploaded_df = upload_csv_file()
    if not uploaded_df.empty:
        df = pd.concat([df, uploaded_df], ignore_index=True)
        print("Batch students loaded successfully.")
    else:
        print("No data loaded.")
    return df

def create_new_student(df):
    """
    Adds a new student to the DataFrame.
    Validates first and surname against provided lists and normalizes text.
    Allows typing 'exit' to return to the main menu.
    """
    while True:
        first_name = input("Enter student's first name (or 'exit' to return to the main menu): ").capitalize()
        if first_name.lower() == 'exit':
            print("Returning to the main menu.")
            return df
        if first_name in first_names_set:
            break
        print("Invalid first name. Please try again.")

    while True:
        surname = input("Enter student's surname (or 'exit' to return to the main menu): ").capitalize()
        if surname.lower() == 'exit':
            print("Returning to the main menu.")
            return df
        if surname in surnames_set:
            break
        print("Invalid surname. Please try again.")

    city = input("Enter the city (or 'exit' to return to the main menu): ").capitalize()
    if city.lower() == 'exit':
        print("Returning to the main menu.")
        return df

    birthday = input("Enter the date of birth (DD/MM/YYYY) (or 'exit' to return to the main menu): ")
    if birthday.lower() == 'exit':
        print("Returning to the main menu.")
        return df

    grades = {}
    for col in df.columns:
        if '_grade' in col:
            while True:
                grade_input = input(f"Enter grade for {col.replace('_grade', '').replace('_', ' ').capitalize()} (or 'exit'): ")
                if grade_input.lower() == 'exit':
                    print("Returning to the main menu.")
                    return df

                # Check if the input is a valid float and within the new valid range (0 to 20)
                if grade_input.replace('.', '', 1).isdigit() and grade_input.count('.') <= 1:
                    grade = float(grade_input)  # Convert the valid input to a float

                    # Check if the grade is within the valid range (0 to 20 in this case)
                    if 0 <= grade <= 20:
                        grades[col] = [grade]
                        break  # Exit the loop if the grade is valid
                    else:
                        print("Grade must be between 0 and 20. Please enter a valid grade.")
                else:
                    print("Invalid input. Please enter a numeric grade.")

    new_student = {
        'first_name': [first_name],
        'surname': [surname],
        'city': [city],
        'birthday': [birthday],
        **grades
    }

    return pd.concat([df, pd.DataFrame(new_student)], ignore_index=True)

def show_current_students(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No students registered at the moment.")
    else:
        print("\nList of students:")
        print(df.to_string(index=False))

def change_record(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No records available to edit or delete.")
        return df

    print("Current records:")
    print(df)

    while True:
        record_index = input("Enter the index of the record you want to edit or delete (or 'exit' to return to the main menu): ").strip()
        if record_index.lower() == 'exit':
            print("Returning to the main menu.")
            return df
        if record_index.isdigit() and 0 <= int(record_index) < len(df):
            record_index = int(record_index)
            break
        print("Invalid index. Please try again.")

    print("Selected record:")
    print(df.iloc[record_index])

    while True:
        action = input("Enter 'edit' to modify or 'delete' to remove the record (or 'exit' to return to the main menu): ").strip().lower()
        if action == 'exit':
            print("Returning to the main menu.")
            return df
        if action in ['edit', 'delete']:
            break
        print("Invalid action. Please enter 'edit', 'delete', or 'exit'.")

    if action == 'edit':
        print("Fields available for modification:")
        for i, col in enumerate(df.columns, start=1):
            print(f"{i}. {col}")

        while True:
            column_choice = input("Enter the number of the field you want to modify (or 'exit' to return to the main menu): ").strip()
            if column_choice.lower() == 'exit':
                print("Returning to the main menu.")
                return df
            if column_choice.isdigit() and 1 <= int(column_choice) <= len(df.columns):
                column_choice = int(column_choice) - 1
                break
            print("Invalid choice. Please try again.")

        column_to_edit = df.columns[column_choice]
        current_value = df.at[record_index, column_to_edit]
        new_value = input(f"Enter the new value for {column_to_edit} (current: {current_value}) (or 'exit' to return to the main menu): ").strip()
        if new_value.lower() == 'exit':
            print("Returning to the main menu.")
            return df

        if new_value:
            if column_to_edit in ['first_name', 'surname', 'city']:
                new_value = new_value.capitalize()
            df.at[record_index, column_to_edit] = new_value
            print("Field successfully updated!")
        else:
            print("No changes made.")

    elif action == 'delete':
        while True:
            confirm = input("Are you sure you want to delete this record? (y/n or 'exit' to return to the main menu): ").strip().lower()
            if confirm == 'exit':
                print("Returning to the main menu.")
                return df
            if confirm in ['y', 'n']:
                break
            print("Invalid input. Please enter 'y', 'n', or 'exit'.")

        if confirm == 'y':
            df = df.drop(index=record_index).reset_index(drop=True)
            print("Record successfully deleted!")
        else:
            print("Deletion canceled.")

    return df

def show_stats(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No records available to display statistics.")
        return

    # Ensure all grade columns are converted to numeric (float)
    grade_columns = [col for col in df.columns if '_grade' in col]
    df[grade_columns] = df[grade_columns].apply(pd.to_numeric, errors='coerce')

def show_stats(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No records available to display statistics.")
        return

    print("\nGeneral statistics:")
    total_students = len(df)
    print(f"Total number of students: {total_students}")

    # Identificar as colunas de notas
    grade_columns = [col for col in df.columns if '_grade' in col]
    if not grade_columns:
        print("No grade columns found.")
        return

    print("\nStatistics by course/unit:")
    for col in grade_columns:
        # Convert grades to numeric (this will handle any non-numeric values by converting them to NaN)
        grades = pd.to_numeric(df[col], errors='coerce')

        # Check for valid (non-NaN) grades
        approved_students = grades[grades >= 10].count()

        # Lista de alunos que passaram e reprovaram
        passed_students = df.loc[grades >= 10, ['first_name', 'surname']]
        failed_students = df.loc[grades < 10, ['first_name', 'surname']]

        print(f"{col.replace('_grade', '').replace('_', ' ').capitalize()}:")
        print(f" - Minimum grade: {grades.min():.2f}")
        print(f" - Average grade: {grades.mean():.2f}")
        print(f" - Maximum grade: {grades.max():.2f}")
        print(f" - Approved students: {approved_students}/{total_students}")

        # List of students who passed and failed
        print("Pass:")
        for _, student in passed_students.iterrows():
            print(f" - {student['first_name'].capitalize()} {student['surname'].capitalize()}")

        print("Fail:")
        for _, student in failed_students.iterrows():
            print(f" - {student['first_name'].capitalize()} {student['surname'].capitalize()}")

    # Estatísticas gerais
    all_grades = pd.concat([pd.to_numeric(df[col], errors='coerce') for col in grade_columns])
    overall_approved = all_grades[all_grades >= 10].count()

    # Calcular alunos que passaram em pelo menos uma disciplina
    students_passed_any_subject = len(df[df[grade_columns].ge(10).any(axis=1)])
    # Calcular alunos que reprovaram todas as disciplinas
    students_failed_all = len(df[df[grade_columns].lt(10).all(axis=1)])

    print("\nOverall approval statistics:")
    print(f" - Total approvals (in all subjects): {overall_approved}")
    print(f" - Students who passed at least one subject: {students_passed_any_subject}/{total_students}")
    print(f" - Students who failed all subjects: {students_failed_all}/{total_students}")

def generate_certificate(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No records available to generate certificates.")
        return

    grade_columns = [col for col in df.columns if '_grade' in col]
    if not grade_columns:
        print("No grade columns found. Cannot calculate final grades.")
        return

    for index, row in df.iterrows():
        student_name = f"{row['first_name']} {row['surname']}"
        grades = pd.to_numeric(row[grade_columns], errors='coerce')
        average_grade = grades.mean()

        if average_grade < 10:
            classification = "Fail"
        elif 10 <= average_grade <= 15:
            classification = "Passed"
        else:
            classification = "Passed with Distinction"

        print("\n--- Final Certificate ---")
        print(f"Name: {student_name}")
        print(f"Average Grade: {average_grade:.2f}")
        print(f"Classification: {classification}")


def export_data(df, file_name='export.xlsx'):
    if df is None or df.empty:  # Check if df is None or empty
        print("No data available for export.")
        return

    df.to_excel(file_name, index=False)
    print(f"Data successfully exported to the file: {file_name}")


def exit_program(df, file_name='database.csv'):
    if df is None or df.empty:  # Check if df is None or empty
        print("No data to save.")
    else:
        save_changes = input("Would you like to save the changes before exiting? (y/n): ").strip().lower()
        if save_changes == 'y':
            df.to_csv(file_name, index=False)
            print(f"Changes successfully saved to the file: {file_name}")
        elif save_changes == 'n':
            print("Changes not saved.")
        else:
            print("Invalid option. Exiting without saving.")

    print("Goodbye!")


import numpy as np
import matplotlib.pyplot as plt

def plot_results(df):
    if df is None or df.empty:  # Check if df is None or empty
        print("No records available to plot results.")
        return

    grade_columns = [col for col in df.columns if '_grade' in col]
    if not grade_columns:
        print("No grade columns found. Cannot plot results.")
        return

    disciplinas = [col.replace('_grade', '').replace('_', ' ').capitalize() for col in grade_columns]
    aprovados = []
    reprovados = []

    for col in grade_columns:
        grades = pd.to_numeric(df[col], errors='coerce')
        aprovados.append((grades >= 10).sum())
        reprovados.append((grades < 10).sum())

    x = np.arange(len(disciplinas))  # Posições no eixo X
    largura = 0.35  # Largura das barras

    # Criação do gráfico de barras
    fig, ax = plt.subplots()
    barras_aprovados = ax.bar(x - largura/2, aprovados, largura, label='Approved', color='green')
    barras_reprovados = ax.bar(x + largura/2, reprovados, largura, label='Failed', color='red')

    # Configurações do gráfico
    ax.set_xlabel('Subjects')
    ax.set_ylabel('Number of Students')
    ax.set_title('Approved and Failed by Subject')
    ax.set_xticks(x)
    ax.set_xticklabels(disciplinas)
    ax.legend()

    # Adicionando rótulos nas barras
    for barra in barras_aprovados:
        altura = barra.get_height()
        ax.text(barra.get_x() + barra.get_width() / 2, altura + 0.5, str(altura), ha='center', color='black')
    for barra in barras_reprovados:
        altura = barra.get_height()
        ax.text(barra.get_x() + barra.get_width() / 2, altura + 0.5, str(altura), ha='center', color='black')

    # Exibe o gráfico
    plt.tight_layout()
    plt.show()

In [ ]:
menu = {
    "1": ("Create new student", create_new_student),
    "2": ("Load batch students (via upload)", load_batch_students_via_upload),
    "3": ("Show current students", show_current_students),
    "4": ("Change some record", change_record),
    "5": ("Show stats about current class", show_stats),
    "6": ("Download data", export_data),
    "7": ("Generate final certificates", generate_certificate),
    "8": ("Plot results by subject", plot_results),
    "-1": ("Exit", exit_program)
}

In [ ]:
def run(menu):
    df = pd.DataFrame()

    print("Hello! Welcome to the Student Management System.")


    while True:
        for cmd, (cmd_msg, _) in menu.items():
            print(f"{cmd} - {cmd_msg}")

        cmd = input()
        if cmd not in menu.keys():
            print("Invalid command")
            continue

        if cmd in ['1', '2', '4', '6']:
            df = menu[cmd][1](df)
        else:
            menu[cmd][1](df)

        if cmd == "-1":
            break

In [ ]:
run(menu)

Hello! Welcome to the Student Management System.
1 - Create new student
2 - Load batch students (via upload)
3 - Show current students
4 - Change some record
5 - Show stats about current class
6 - Download data
7 - Generate final certificates
8 - Plot results by subject
-1 - Exit
